<a href="https://colab.research.google.com/github/Aqsagull99/project_4/blob/main/country_information_cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text]https://countryinformationcards.streamlit.app/

In [ ]:
import streamlit as st
import requests
import pandas as pd
from PIL import Image
from io import BytesIO

# App configuration
st.set_page_config(page_title="Country Information Cards", layout="wide")

# CSS styling
st.markdown("""
    <style>
    .country-card {
        border-radius: 10px;
        padding: 20px;
        margin-bottom: 20px;
        background-color: #f8f9fa;
        box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
        transition: 0.3s;
    }
    .country-card:hover {
        box-shadow: 0 8px 16px 0 rgba(0,0,0,0.2);
    }
    .country-flag {
        height: 120px;
        width: auto;
        object-fit: contain;
        margin-bottom: 15px;
        border: 1px solid #ddd;
    }
    .country-name {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 10px;
        color: #2c3e50;
    }
    .info-label {
        font-weight: bold;
        color: #3498db;
    }
    .stSelectbox div[data-baseweb="select"] > div {
        padding: 10px;
    }
    </style>
    """, unsafe_allow_html=True)

# Cache country data to avoid repeated API calls
@st.cache_data(ttl=3600)
def get_countries_data():
    try:
        response = requests.get("https://restcountries.com/v3.1/all") #  Rest Countries API
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching country data: {e}")
        return None

# Main app function
def main():
    st.title("🌍 Country Information Cards")
    st.markdown("Explore detailed information about countries around the world.")

    # Get country data
    countries_data = get_countries_data()
    if not countries_data:
        return

    # Create a list of country names for selection
    country_names = sorted([country.get("name", {}).get("common", "Unknown") for country in countries_data])

    # Sidebar filters
    st.sidebar.title("Filters")
    selected_countries = st.sidebar.multiselect(
        "Select Countries",
        country_names,
        default=["United States", "Canada", "Japan", "Germany", "Brazil"]
    )

    region_list = sorted(list(set([country.get("region", "Unknown") for country in countries_data])))
    selected_region = st.sidebar.selectbox("Filter by Region", ["All"] + region_list)

    # Filter countries based on selections
    filtered_countries = []
    for country in countries_data:
        name = country.get("name", {}).get("common", "")
        region = country.get("region", "")

        name_match = not selected_countries or name in selected_countries
        region_match = selected_region == "All" or region == selected_region

        if name_match and region_match:
            filtered_countries.append(country)

    if not filtered_countries:
        st.warning("No countries match your filters. Please adjust your selection.")
        return

    # Display country cards
    cols = st.columns(3)
    for i, country in enumerate(filtered_countries):
        with cols[i % 3]:
            with st.container():
                # Country card container
                st.markdown(f'<div class="country-card">', unsafe_allow_html=True)

                # Country name
                official_name = country.get("name", {}).get("official", "")
                common_name = country.get("name", {}).get("common", "Unknown")
                st.markdown(f'<div class="country-name">{common_name}</div>', unsafe_allow_html=True)

                # Country flag
                flag_url = country.get("flags", {}).get("png", "")
                if flag_url:
                    try:
                        response = requests.get(flag_url)
                        flag_img = Image.open(BytesIO(response.content))
                        st.image(flag_img, caption=f"Flag of {common_name}", use_container_width=True)
                    except:
                        st.write("Flag not available")

                # Basic information
                st.markdown(f"""
                    <p><span class="info-label">Official Name:</span> {official_name}</p>
                    <p><span class="info-label">Capital:</span> {', '.join(country.get('capital', ['N/A']))}</p>
                    <p><span class="info-label">Region:</span> {country.get('region', 'N/A')}</p>
                    <p><span class="info-label">Subregion:</span> {country.get('subregion', 'N/A')}</p>
                    <p><span class="info-label">Population:</span> {country.get('population', 'N/A'):,}</p>
                    <p><span class="info-label">Area:</span> {country.get('area', 'N/A'):,} km²</p>
                """, unsafe_allow_html=True)

                # Languages
                languages = country.get("languages", {})
                if languages:
                    lang_list = ", ".join(languages.values())
                    st.markdown(f'<p><span class="info-label">Languages:</span> {lang_list}</p>', unsafe_allow_html=True)

                # Currency
                currencies = country.get("currencies", {})
                if currencies:
                    currency_info = []
                    for code, details in currencies.items():
                        currency_info.append(f"{details.get('name', '')} ({details.get('symbol', '')})")
                    st.markdown(f'<p><span class="info-label">Currency:</span> {", ".join(currency_info)}</p>', unsafe_allow_html=True)

                # Google Maps link
                maps = country.get("maps", {}).get("googleMaps", "")
                if maps:
                    st.markdown(f'<a href="{maps}" target="_blank">View on Google Maps</a>', unsafe_allow_html=True)

                st.markdown('</div>', unsafe_allow_html=True)

if __name__ == "__main__":
    main()



